In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [66]:
path = 'F:/Books/Machine Learning/DataSets/Titanic'
raw_data = pd.read_csv('all_data.csv')
train_label = pd.read_csv('labels.csv')
sub_data = pd.read_csv(path + '/gender_submission.csv')


In [67]:
raw_data = pd.concat([raw_data , pd.get_dummies(raw_data.Sex,drop_first=True,prefix='Sex'),
pd.get_dummies(raw_data.Embarked,prefix='Embarked')],axis=1)
raw_data.drop(['Embarked','Sex'],axis=1,inplace=True)
raw_data.head()


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,1,0,0,1
1,1,38.0,1,0,71.2833,0,1,0,0
2,3,26.0,0,0,7.9250,0,0,0,1
3,1,35.0,1,0,53.1000,0,0,0,1
4,3,35.0,0,0,8.0500,1,0,0,1


In [68]:
# Seperating data
train_data = raw_data.iloc[:891]
test_data = raw_data.iloc[891:]

# Remove outliers
drop_obser = []
drop_obser.extend(train_data.SibSp[train_data.SibSp > 5].index.tolist())
drop_obser.extend(train_data.Parch[train_data.Parch > 4].index.tolist())
train_data.drop(set(drop_obser),inplace=True)
train_label.drop(set(drop_obser),inplace=True)

In [69]:
from xgboost              import XGBClassifier
from sklearn.svm          import SVC
from sklearn.ensemble     import RandomForestClassifier
from sklearn.ensemble     import AdaBoostClassifier
from sklearn.ensemble     import GradientBoostingClassifier
from sklearn.metrics      import r2_score, accuracy_score, mean_squared_error
from sklearn.pipeline     import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix 

In [70]:
x_train, x_test, y_train, y_test = train_test_split(train_data,
                                                    train_label,
                                                   test_size=.30,
                                                   random_state= 42,
                                                   shuffle=True)

In [71]:
df = pd.DataFrame()
svc = SVC()
svc.fit(x_train,y_train)
y_pred_svc = svc.predict(x_test)
df = pd.concat([df , pd.DataFrame(y_pred_svc,columns=['SVC'])],axis=1)

adb = AdaBoostClassifier()
adb.fit(x_train,y_train)
y_pred_adb = adb.predict(x_test)
df = pd.concat([df , pd.DataFrame(y_pred_adb,columns=['ADB'])],axis=1)

best_params_1 = {'subsample': 0.6, 
                 'reg_lambda': 1,
                 'reg_alpha': 0, 
                 'n_estimators': 500, 
                 'min_child_weight': 3, 
                 'max_depth': 7,
                 'learning_rate': 0.01, 
                 'gamma': 1.5, 
                 'colsample_bytree': 0.8}
xgb_1 = XGBClassifier(**best_params_1)
xgb_1.fit(x_train,y_train)
y_pred_xgb1 = xgb_1.predict(x_test)
df = pd.concat([df , pd.DataFrame(y_pred_xgb1,columns=['XGB_1'])],axis=1)


best_params_2 ={ 'subsample': 0.55, 
                'reg_lambda': 1, 
                'reg_alpha': 0, 
                'n_estimators': 520,
                'min_child_weight': 3,
                'max_depth': 6,
                'learning_rate': 0.03,
                'gamma': 1.2,
                'colsample_bytree': 0.9}


xgb_2 = XGBClassifier(**best_params_2)
xgb_2.fit(x_train,y_train)
y_pred_xgb2 = xgb_2.predict(x_test)
df = pd.concat([df , pd.DataFrame(y_pred_xgb2,columns=['XGB_2'])],axis=1)

gdb = GradientBoostingClassifier()
gdb.fit(x_train,y_train)
y_pred_gdb = gdb.predict(x_test)
df = pd.concat([df , pd.DataFrame(y_pred_adb,columns=['GRB'])],axis=1)


In [72]:
print(classification_report(y_test,y_pred_xgb2))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       165
           1       0.82      0.76      0.79        99

    accuracy                           0.85       264
   macro avg       0.84      0.83      0.84       264
weighted avg       0.85      0.85      0.85       264



In [74]:
best_params_2 ={ 'subsample': 0.55, 
                'reg_lambda': 1, 
                'reg_alpha': 0, 
                'n_estimators': 520,
                'min_child_weight': 3,
                'max_depth': 6,
                'learning_rate': 0.03,
                'gamma': 1.2,
                'colsample_bytree': 0.9}


xgb_2 = XGBClassifier(**best_params_2)
xgb_2.fit(train_data,train_label)
y_pred_ = xgb_2.predict(test_data)
